In [7]:
# ======================================================================
# == load the libraries ==
# ======================================================================

from stardist.data import test_image_nuclei_2d
from stardist.plot import render_label
from stardist.models import StarDist2D
from shapely.geometry import Polygon, Point
import matplotlib.image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import pandas as pd
import numpy as np
import seaborn as sns
import PIL
import tifffile
import sys
import pickle


In [ ]:
# ======================================================================
# == definition of custom functions ==
# ======================================================================

from custom_functions import custom_normalize

In [10]:
# ======================================================================
# == processing ==
# ======================================================================

# --- read in the input image ---

# img hires
PIL.Image.MAX_IMAGE_PIXELS = 1718032108
img = tifffile.imread("../../../data/Mouse_Embryo/Visium_HD_Mouse_Embryo_tissue_image.btf")
img = img[:,:,0:3]

# image type
hist_img = "H&E"

# stardist parameters
# prob_thresh
# nms_thresh

In [11]:
# --- run custom normalization ---

normalized_img = custom_normalize(img)


In [12]:
# --- select approprote stardist model ---

if hist_img == "H&E":
    model_stardist = "2D_versatile_he"
elif hist_img == "IF":
    model_stardist = "2D_versatile_fluo"

model = StarDist2D.from_pretrained(model_stardist)

Found model '2D_versatile_he' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.692478, nms_thresh=0.3.


In [13]:
# --- run the model ---

# here is running with default parameters for prob_thresh and nms_thresh
labels, polys = model.predict_instances_big(
    normalized_img, axes='YXC', block_size=4096, min_overlap=128, context=128,
    normalizer=None, # n_tiles=(4,4,1),
    prob_thresh = .05)

effective: block_size=(4096, 4096, 3), min_overlap=(128, 128, 0), context=(128, 128, 0)


functional.py (238): The structure of `inputs` doesn't match the expected structure.
Expected: ['input']
Received: inputs=Tensor(shape=(1, 256, 256, 3))
  0%|          | 0/56 [00:00<?, ?it/s]functional.py (238): The structure of `inputs` doesn't match the expected structure.
Expected: ['input']
Received: inputs=Tensor(shape=(1, None, None, 3))
100%|██████████| 56/56 [13:38<00:00, 14.61s/it]


In [ ]:
# ======================================================================
# == save the output ==
# ======================================================================

with open("../../../data/Mouse_Embryo/nuclei_polys.pkl", "wb") as f:  # open a text file
    pickle.dump(polys, f)